# Import tokenization patterns from etTenTen patterns

The file `ettenten_patterns.py` is downloaded from [https://github.com/kristiinavaik/ettenten-eeltootlus](https://github.com/kristiinavaik/ettenten-eeltootlus).

In [1]:
from ettenten_patterns import PATTERNS

In [2]:
# NBVAL_IGNORE_OUTPUT
patterns_and_replacements = []

for pattern, replacement in PATTERNS:
    if isinstance(replacement, str):
        patterns_and_replacements.append((pattern.pattern.strip(), replacement))
    else:
        print (pattern, replacement)
# üks musteritest (PATT_16) on hoopis funktsioon

<function PATT_16 at 0x7fc26941a620> None


In [3]:
len(patterns_and_replacements)

89

In [4]:
import re
# vaatame millised tagid esinevad mustrite ja asenduste sees
pattern_tag_set = set()
replace_tag_set = set()
for p, r in patterns_and_replacements:
    for tag in re.finditer('<.*?>', p):
        pattern_tag_set.add(tag.group())
    for tag in re.finditer('<.*?>', r):
        replace_tag_set.add(tag.group())
print(sorted(pattern_tag_set))
print(sorted(replace_tag_set))

['<+/>', '</doc>', '</ignore>', '</p>', '</s>', '<>', '<\\+/>', '<\\+[^:]+:<\\+/>', '<]+)</s>', '<a>', '<b>', '<ignore>', '<p[^>', '<s>']
['<+/>', '</ignore>', '</s>', '<ignore>', '<s>']


In [5]:
# mustrid, mis sisaldavad <+/>
for p, r in patterns_and_replacements:
    if '<+/>' in p:
        print(p)

(\d+)\s?(±)\s?((<+/>)?\d+)


In [6]:
# asendused, mis sisaldavad '<+/>'
for p, r in patterns_and_replacements:
    if '<+/>' in r:
        print('pattern:')
        print(p)
        print('replacement:', r)
        print('--------------------------------------------------------------------------')

pattern:
(\d+)\s(\.\.\.)\s(\d+)
replacement: \1<+/>\2<+/>\3
--------------------------------------------------------------------------
pattern:
((\s|\s?[-–+]|<\+/>)\d+±?)\s((?![0-9]+:)\d+(,\d+)?(\s|<\+/>|\.)(?!aasta))
replacement: \1<+/>\3
--------------------------------------------------------------------------
pattern:
(\d+<\+/>\d+)\s
    (\d+(<\+/>\d+(<\+/>\d+)?)?)
replacement: \1<+/>\2
--------------------------------------------------------------------------
pattern:
(\d+)\s?(±)\s?((<+/>)?\d+)
replacement: \1<+/>\2<+/>\3
--------------------------------------------------------------------------
pattern:
([0-9]+)\s
    ([–-])\s?
    ((?![0-9]+:)[0-9]+%?)
replacement: \1<+/>\2<+/>\3
--------------------------------------------------------------------------
pattern:
(\s[A-ZÕÜÖÄ][A-ZÕÜÖÄa-zõüöä]*)   # mistahes mittetühik
    \s         # kuni tühik
    (&amp;)\s  # ampersand ja kogu järgnev tühikuni
    ([A-ZÕÜÖÄ])
replacement: \1<+/>\2<+/>\3
-----------------------------------------

In [7]:
# mustrid, mis sisaldavad 'ignore'
for p, r in patterns_and_replacements:
    if 'ignore' in p:
        print(p)
        print('-------------------------------------------------------')

(<ignore>\s)?
    (\(\s
        (
            ([A-ZÕÜÖÄa-zõüöä0-9,-<>[]/]{1,3}|\+)\s?
        )*
    \s\))
    (\s</ignore>)?
-------------------------------------------------------
(<ignore>[^>]+?)
        <ignore>\s([^>]+?)\s</ignore>
    ([^>]+?</ignore>)
-------------------------------------------------------
(?P<a>\s</ignore>\s,\s)
    (?P<b><ignore>)
-------------------------------------------------------
([0-9a-zõüöäA-ZÕÜÖÄ])(</ignore>)
-------------------------------------------------------
(?P<a>\s</ignore>\sja\s)
    (?P<b><ignore>)
-------------------------------------------------------
(<ignore>\s\(\s([A-ZÕÜÖÄa-zõüöä]+\s)+\.\s)</s>\s<s>\s<ignore>\s(\d+\s\)\s)</ignore>\s(</ignore>)
-------------------------------------------------------
(<ignore>((?!<ignore>).)+)<ignore>\s(((?!</ignore>).)+)\s</ignore>(((?!</ignore>).)+</ignore>)
-------------------------------------------------------
(?P<a>\s</ignore>\s)
    (?P<b><ignore>)
--------------------------------------------------

In [8]:
# asendused, mis sisaldavad 'ignore'
for p, r in patterns_and_replacements:
    if 'ignore' in r:
        print('pattern:')
        print(p)
        print('replacement:', r)
        print('--------------------------------------------------------------------------')

pattern:
(\([^)]+\))?
    (\(              # viide algab suluga
        (            # viite sees võivad olla
            [^()]  # sümbolid, mis pole sulud
        )+    # rekursiivselt analoogiline struktuur
        
    \s?\d+,?\s?[a-zõüöäA-ZÕÜÖÄ%'"\]!]*  # arv (aasta)
    (/\d+)?[-']?\s               # valikuline kaldkriipsuga aastaarv
    (                       # valikulise grupi 2 algus
        :\s                 # koolon
        \d+                 # arvud, nt lk nr
        ([-–]\d+[\.,]+)\s     # kuni-märk ja arvud, nt lk 5-6
    )?                      # grupi 2 lõpp
    -?\w*\s?\.?\s?                     # lõpus tühik
        # viide lõpeb aastaarvuga
    \))              # viide lõpeb suluga
replacement: <ignore> \2 </ignore>
--------------------------------------------------------------------------
pattern:
(<ignore>\s)?
    (\(\s
        (
            ([A-ZÕÜÖÄa-zõüöä0-9,-<>[]/]{1,3}|\+)\s?
        )*
    \s\))
    (\s</ignore>)?
replacement: <ignore> \2 </ignore>
-------

In [9]:
# nimekiri kõikidest asendustest
for p, r in patterns_and_replacements:
    print(r)

\1 \2 </s> <s> \3
\1\2 </s> <s> \3
\2\1
\2\1
\1
\1
\1\3\2\4
\1\3\4
\1\3\2
\1\3\4
\1\2
\1\3
\1\2\3\4
\2\1 \4
\1.\2
\1\2
\1\2\3
\1.\2
<ignore> \2 </ignore>
<ignore> \2 </ignore>
\1\2\3
\1<+/>\2<+/>\3
\1<+/>\3
\1<+/>\2
\1<+/>\2<+/>\3
\1<+/>\2<+/>\3
\1\2
\1.\2.\3
\1<+/>\2<+/>\3
\1\2\3
\1\2\3
\1<+/>\4
\1<+/>\4
\1\2
\1<+/>\2<+/>\3
\1\2<+/>\3<+/>\4\5
\1\2
\1.\2.<+/>\3
\1.<+/>\2
\1 \2
\1\2\3\4\5
\1<+/>:<+/>\3
,
\1 \2
 ja
\1\2
\1 \2
\1<+/>\2<+/>\3
\1\3\4
\1 <ignore>\2 </ignore>
\1 \2
\1<ignore> \2</ignore> 
\1,\2,\3
\1\3\4
\1\3\5
\1<+/>\2<+/>\3
\1<+/>\2
\1<+/>\2<+/>\3
\1<+/>\2
\1<+/>:<+/>\3
\1\3
\1\2
\1\2\3
\1\2\3
\1\3
\1\2
\1\3
\1<+/>\2/\3
\1 <ignore>\2</ignore> \3
<ignore> \1 </ignore>

\1 \2
\1<ignore> \2 <ignore>\3
\1<+/>\2
\1\2\3[\4]\5
\1\2\3[\4]\5
\1[\2]\3
\1 \2
\1 \2
\1<ignore> \2 </ignore>\3
\1<ignore> \2 </ignore>\3
\1.\2.\3
\1.\2.\3
\1:\2\3
\1\2\3

\1
\1
\1


In [10]:
# asendused, mis sisaldavad '<+/>'
len([r for p, r in patterns_and_replacements if '<+/>' in r])

21

In [11]:
# asendused, mis sisaldavad 'ignore'
len([r for p, r in patterns_and_replacements if 'ignore' in r])

9

In [12]:
# asendused, mis sisaldavad '<+/>' ja 'ignore'
len([r for p, r in patterns_and_replacements if '<+/>' in r and 'ignore' in r])

0

In [13]:
# mustrid, mis sisaldavad '<s>' või '</s>'
len([p for p, r in patterns_and_replacements if '<s>' in p or '</s>' in p])

35

In [14]:
import rstr
import sys
sys.modules['rstr.xeger'].STAR_PLUS_LIMIT = 2

def examine_pattern(n, line):
    '''abimees asendusreeglite uurimiseks
    mõne reegliga rstr ei tööda
    n - reegli number
    line - suvaline string, millele reeglit rakendada
    '''
    r, s = PATTERNS[n]
    print('pattern')
    print(PATTERNS[n][0].pattern.strip())
    compact_pattrern = re.sub('\s*(#.*)?' , '',  PATTERNS[n][0].pattern)
    separator = '-'*len(compact_pattrern)
    print(separator)
    print('compact_pattern')
    print(compact_pattrern)
    print(separator)
    print('some examples')
    print(re.sub('\s', ' ', rstr.xeger(compact_pattrern)))
    print(re.sub('\s', ' ', rstr.xeger(compact_pattrern)))
    print(re.sub('\s', ' ', rstr.xeger(compact_pattrern)))
    print(separator)
    print('substitution')
    print(PATTERNS[n][1])
    print(separator)
    result = r.sub(s, line)
    print('line:  ', repr(line))
    print('result:', repr(result))
    print('result != line:', result != line)
    print(separator)
    print('line for testset')
    print("('" + line + "', '" + result + "'), #" + str(n))

In [15]:
# NBVAL_IGNORE_OUTPUT
examine_pattern(74,  '–86 70,9.')

pattern
([-–][0-9]+)\s([0-9]+<\+/>)
---------------------------
compact_pattern
([-–][0-9]+)\s([0-9]+<\+/>)
---------------------------
some examples
–7 0<+/>
-58 6<+/>
-58 4<+/>
---------------------------
substitution
\1<+/>\2
---------------------------
line:   '–86 70,9.'
result: '–86 70,9.'
result != line: False
---------------------------
line for testset
('–86 70,9.', '–86 70,9.'), #74


In [16]:
testset = [
           ('a..Aa', 'a .. </s> <s> Aa'), #0 kaks punkti on lause piir
           ('a .Aa', 'a . </s> <s> Aa'), #1 tühik ja punkt on lause piir
           ('</s> <s> ) . ', ') . </s> <s> '), #2 ümarsulg kinni lause alguses kuulub eelmise lause lõppu
           ('</s> <s> ] ', '] </s> <s> '), #3 nurksulg kinni lause alguses kuulub eelmise lause lõppu
           ('</s> <s> ) , ', ') , '), #4 kui lause algab sulu ja komaga, siis see pole uus lause
           ('</s> <s> </s> <', '</s> <'), #5 tühja lause kustutamine
           (' a . Aa </s> <s> a', ' a . </s> <s> Aa a'), #6 suurtähega sõna lause lõpus pärast punkti on järgmise lause algus
           (' Hr . Aa </s> <s>', ' Hr. Aa'), #7 tiitel, punkt ja nimi ei ole lause lõpp
           (' a . Aa , </s> <s>', ' a . </s> <s> Aa ,'), #8 suurtäht ja koma pärast punkti on järgmise lause algus
           (' a - e ', ' a-e '), #9 käändelõpp sõne külge
           (' so . </s> <s>', ' so.'), #10 'so.' ei ole lause lõpp
           (' vrd . </s> <s>', ' vrd.'), #11 'vrd.' ei ole lause lõpp
           (' ibid . a </s> <s> Lk. 1', ' ibid. a Lk. 1'), #12 ibid. ei ole lause lõpp
           ('</s> a <s>', 'a </s> <s>'), #13 kui midagi on kahe lause vahel, siis see on eelmise lause lõpp
           ('1 . </s> <s> a. ', '1.a. '), #14 1.a.
           (' hrl . </s> <s>', ' hrl.'), #15 'hrl.' ei ole lause lõpp
           (' Jr . </s> <s> , ', ' Jr. , '), #16 'Jr.' komaga ei ole lause lõpp
           ('1 . </s> <s> aasta', '1. aasta'), #17 järgarv ja aasta käivad kokku
           ('(a)(b , 2000 : lk 1-8. c 2002 . )', '<ignore> (b , 2000 : lk 1-8. c 2002 . ) </ignore>'), #18 viiteid ignoreeritakse
           ('<ignore>  ( + ) </ignore>', '<ignore>  <ignore> ( + ) </ignore>'), #19
           ('<ignore> a <ignore> a </ignore>a</ignore>', '<ignore> a aa</ignore>'), #20 pesastatud <ignore>
           ('89 ... 4', '89<+/>...<+/>4'), #21 kolme punktiga eraldatud arvud kuuluvad kokku
           ('<+/>61± 8,5.', '<+/>61±<+/>8,5.'), #22 sellele rakendub ka #24
           ('53<+/>60 1', '53<+/>60<+/>1'), #23
           ('9±92', '9<+/>±<+/>92'), #24
           ('1 -34', '1<+/>-<+/>34'), #25
           ('55 %a ', '55%a '), #26
           ('6 .66 .27 ', '6.66.27 '), #27
           (' A &amp; B', ' A<+/>&amp;<+/>B'), #28
           ('2017. a. ', '2017.a. '), #29
           (' 3 . 0 megavatti', ' 3.0 megavatti'), #30
           ('4 kcal ', '4<+/>kcal '), #31
           ('<+/>20,17. a. ', '<+/>20,17.<+/>a. '), #32
           (' § -p ', ' §-p '), #33
           ('35- 6%', '35<+/>-<+/>6%'), #34
           ('9.- 5', '9.<+/>-<+/>5'), #35
           ('2017 a ', '2017a '), #36
           ('V .I. Lenin', 'V.I.<+/>Lenin'), #37
           ('V . I . Lenin', 'V . I.<+/>Lenin'), #38
           ('', None),#39 on funktsioon
           ('5<+/>8-st', '5 8-st'), #40
           ('8 ... 3 , 1', '8...3,1'), #41
           ('>5 :13', '>5<+/>:<+/>13'), #42 avaldis jagamisega on üks sõne
           (' </ignore> , <ignore>', ','), #43 üksikut koma ei ignoreeri
           ('G</ignore>', 'G </ignore>'), #44 üksikut sümbolit pärast <ignore> tuleb ka ignoreerida
           (' </ignore> ja <ignore>', ' ja'), #45 üksikut 'ja' ei ignoreei
           ('<+/> 1', '<+/>1'), #46 '<+/>1' järel ei ole tühik
           ('1234<+/>56.', '1234 56.'), #47
           ('71 x 3', '71<+/>x<+/>3'), #48 korrutustehe on üks sõne
           ('( 37-50 . </s> <s> aasta', '( 37-50. aasta'), #49 järgarvu punkt ei lõpeta lauset
           ('u ( XD 4. )', 'u <ignore> ( XD 4. ) </ignore>'), #50
           (')  +4', ') +4'), #51 topelttühiku eemaldus
           ('Bla ( Lenin , Marx ) ', 'Bla <ignore> ( Lenin , Marx ) </ignore> '), #52 viite ignoreerimine
           ('31 , 4<+e9:<+/>3 , 79', '31,4<+e9:<+/>3,79'), #53
           ('<ignore> ( V d . </s> <s> <ignore> 3 ) </ignore> </ignore>', '<ignore> ( V d . 3 ) </ignore>'), #54 pesastatud <ignore>
           ('<ignore> ( V d . </s> <s> <ignore> 3 ) </ignore> </ignore>', '<ignore> ( V d . </s> <s> 3 ) </ignore>'), #55 sarnane eelmisele
           ('9 + -7', '9<+/>+<+/>-7'), #56 avaldis on üks sõne
           (' x 2', ' x<+/>2'), #57 korrutamismärk kuulub arvuga kokku
           (' h = -4', ' h<+/>=<+/>-4'), #58 avaldis on üks sõne
           (' B 30 ', ' B<+/>30 '), #59 suurtäht ja arv kuuluvad kokku
           ('> 6 :83 ', '> 6<+/>:<+/>83 '), #60 kordab #42
           (' lüh. </s> <s>', ' lüh.'), #61 punkt lühendi lõpus ei ole lause lõpp
           (' st . </s> <s>', ' st.'), #62 punkt lühendi lõpus ei ole lause lõpp
           (' s . </s> <s> t. ', ' s.t. '), #63 punkt lühendi lõpus ei ole lause lõpp
           (' tn. </s> <s> 0', ' tn. 0'), #64 punkt lühendi lõpus ei ole lause lõpp
           (' Koost . </s> <s>', ' Koost.'), #65 punkt lühendi lõpus ei ole lause lõpp
           (' Mrs . </s> <s>', ' Mrs.'), #66 punkt lühendi lõpus ei ole lause lõpp
           (' tlk. </s> <s>', ' tlk.'), #67 punkt lühendi lõpus ei ole lause lõpp
           ('27 km/ h ', '27<+/>km/h '), #68 arv koos murrulise mõõtühikuga on üks sõne
           ('</s> <s> 1 </s> </p>', '</s> <s> <ignore> 1 </ignore> </s> </p>'), #69 ühest arvust koosnevat lauset paragrahvi lõpus ignoreeritakse
           ('[ 1 ]', '<ignore> [ 1 ] </ignore>'), #70 nurksulgudes arvu ignoreeritakse
           (' </ignore> <ignore>', ''), #71 kaks järjestikust ignore on üks ignore
           ('28.detsemb', '28. detsemb'), #72 kuupäev koos kuu nimega on kaks eraldi sõne
           ('<px> <s> 5 . </s> </p>', '<px> <ignore> <s> 5 . </s> <ignore> </p>'), #73 paragrahvi? ainsat (järg)arvust koosnevat lauset ignoreeritakse
           ('-2 4<+/>', '-2<+/>4<+/>'), #74
           (' nikita . </s> <s> h [ -at- ] kreml.ru', ' nikita.h[-at-]kreml.ru'), #75 punkt meiliaadressis pole lause lõpp
           (' nikita . h </s> <s> [ -at- ] kreml.ru', ' nikita.h[-at-]kreml.ru'), #76 -at- meiliaadressis ei alusta uut lauset
           (' nikita.h [ -at- ] kreml.ru', ' nikita.h[-at-]kreml.ru'), #77 meiliaadress on üks sõne
           (' bla</ignore', ' bla </ignore'), #78 </ignore> märgendit eraldab tühik
           ('ignore>bla ', 'ignore> bla '), #79 <ignore> märgendit eraldab tühik
           ('<s> 99% </s> </p>', '<s> <ignore> 99% </ignore> </s> </p>'), #80 lühikese lause sisu paragrahvi lõpus ignoreeritakse
           ('<s> 2. </s>', '<s> <ignore> 2. </ignore> </s>'), #81 lühikese lause sisu ignoreeritakse
           (' /www . </s> <s> bla . </s> <s> ee', ' /www.bla.ee'), #82 veebiaadress on üks sõne, mitte mitu lauset
           (' /www . </s> <s> bla . ee', ' /www.bla.ee'), #83 veebiaadress on üks sõne, mitte mitu lauset
           ('http : /www ', 'http:/www '), #84 veebiaadress on üks sõne
           ('www? / x ', 'www?/x '), #85
           ('<p> <s> </s> </p>\n', ''), #86 tühi paragrahv enne reavahetust kustutatakse
           (' <s> </s> ', ' '), #87 tühi lause kustutatakse
           ('<p??> </p> </doc>', '</doc>'), #88 tühi paragrahv dokumendi lõpus kustutatakse
           ('</doc>\n', '</doc>'), #89 dokumendi lõpus reavahetus eemaldatakse
          ]

# PATT_4_2 on listist PATTERNS välja jäetud
# PATT_48 on topelt
# PATT_3_4 sisaldab (<+/>), aga vist peaks olema (<\+/>)

# test PATTERNS
for i, ((r, s), (line_1, line_2)) in enumerate(zip(PATTERNS, testset)):
    if callable(r):
        continue
    result = r.sub(s, line_1)
    assert line_2 == result, 'rule #' +str(i)+ '\nexpected: ' + repr(line_2) + '\nresult:   '+repr(result)